In [1]:
import mailcom.inout
import mailcom.parse
import pandas as pd

c:\Users\Felix\miniconda3\envs\mailcom\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import files from /data/in

io = mailcom.inout.InoutHandler("../mailcom/test/data")
io.list_of_files()

# create pseudonymization object
ps = mailcom.parse.Pseudonymize()
ps.init_spacy("fr")
ps.init_transformers()

Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# loop over mails and pseudonymize them
out_list = []
for file in io.email_list:
    print("Parsing input file {}".format(file))
    text = io.get_text(file)
    # after this function was called, the email metadata can be accessed via io.email_content
    # the dict already has the entries content, date, attachments, attachment type
    email_dict = io.email_content.copy()
    text = io.get_html_text(text)
    if not text:
        continue
    # Test functionality of Pseudonymize class
    output_text = ps.pseudonymize(text)
    email_dict["pseudo_content"] = output_text
    out_list.append(email_dict)


Parsing input file C:\Users\Felix\Documents\GitHub\mailcom\mailcom\test\data\Bonjour Agathe.eml
Parsing input file C:\Users\Felix\Documents\GitHub\mailcom\mailcom\test\data\Re reunião agendada para o dia 24 de abril 2024-04-17T17_39_49+02 00.eml
Parsing input file C:\Users\Felix\Documents\GitHub\mailcom\mailcom\test\data\Re_ purismo.html
Parsing input file C:\Users\Felix\Documents\GitHub\mailcom\mailcom\test\data\Re_ ¡Voy el 24!.html


In [4]:
# write output to pandas df
df = pd.DataFrame(out_list)

In [5]:
# print results
for idx, mail in df.iterrows():
    print("Email", idx)
    print("Original Text:\n", mail["content"])
    print("Pseudonymized Text:\n", mail["pseudo_content"])	

Email 0
Original Text:
 J'espère que tu vas bien! Je voulais partager avec toi quelques photos de mon dernier voyage!
[Inline-Bild]

[Inline-Bild]

À bientôt,

Pierre

Pseudonymized Text:
 J'espère que tu vas bien! Je voulais partager avec toi quelques photos de mon dernier voyage!
[Inline-Bild]

[Inline-Bild]

À bientôt,

Claude

Email 1
Original Text:
 <html><head></head><body><div class="ydp9a5bdb1ayahoo-style-wrap" style="font-family:Helvetica Neue, Helvetica, Arial, sans-serif;font-size:16px;"><div></div>
        <div><div><p style="border: 0px solid rgb(227, 227, 227); margin: 1.25em 0px; color: rgb(13, 13, 13); font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Ubuntu, Cantarell, Noto Sans, sans-serif, Helvetica Neue, Arial, Apple Color Emoji, Segoe UI Emoji, Segoe UI Symbol, Noto Color Emoji; font-size: 16px;">Olá Lino,</p><p style="border: 0px solid rgb(227, 227, 227); margin: 1.25em 0px; color: rgb(13, 13, 13); font-family: ui-sans-serif, system-ui, -appl